# Import Required Libraries

In [21]:
import random
from sympy import isprime, mod_inverse, gcd

# Prime Number Generation

In [22]:
def generate_large_prime(bits):
    while True:
        num = random.getrandbits(bits)  # Generate a random number with 'bits' bits
        if isprime(num):  # Check if the generated number is a prime number
            return num  # Return the prime number


# Key Generation

In [23]:
def generate_keys(bits=1024):
    p = generate_large_prime(bits)  # Generate a large prime number p
    q = generate_large_prime(bits)  # Generate a large prime number q
    
    n = p * q  # Calculate n = p * q
    phi = (p - 1) * (q - 1)  # Calculate the totient function phi = (p - 1) * (q - 1)
    
    # Choose e randomly between 1 and phi
    while True:
        e = random.randrange(1, phi)  # Randomly choose e in the range [1, phi)
        if gcd(e, phi) == 1:  # Ensure e and phi are coprime
            break  # Exit the loop if they are coprime
    
    d = mod_inverse(e, phi)  # Calculate the modular inverse of e modulo phi
    
    return ((e, n), (d, n))  # Return the public key (e, n) and private key (d, n)


# Encryption

In [24]:
def encrypt(plaintext, public_key):
    e, n = public_key  # Unpack the public key
    plaintext_bytes = plaintext.encode('utf-8')  # Convert plaintext to bytes
    plaintext_int = int.from_bytes(plaintext_bytes, byteorder='big')  # Convert bytes to integer
    ciphertext_int = pow(plaintext_int, e, n)  # Perform modular exponentiation
    return ciphertext_int  # Return the ciphertext as an integer


# Decryption

In [25]:
def decrypt(ciphertext, private_key):
    # Unpack the private key tuple into d (private exponent) and n (modulus)
    d, n = private_key
    
    # Decrypt the ciphertext using modular exponentiation
    plaintext_int = pow(ciphertext, d, n)
    
    # Convert the decrypted integer into bytes
    # Calculate the number of bytes needed based on the bit length of the integer
    # Convert the integer to bytes using big-endian byte order
    plaintext_bytes = plaintext_int.to_bytes((plaintext_int.bit_length() + 7) // 8, byteorder='big')
    
    # Try to decode the bytes into a UTF-8 string
    try:
        # If successful, return the decoded plaintext string
        return plaintext_bytes.decode('utf-8')
    # Handle the case where decoding fails due to invalid byte sequences
    except UnicodeDecodeError:
        # Return the decrypted integer representation if decoding fails
        return plaintext_int


# Putting It All Together

In [26]:
def main():
    # Generate public and private keys
    public_key, private_key = generate_keys(bits=512)
    
    # Get message from user
    message = input("Enter the message to encrypt: ")  # Prompt the user for input
    print("Original Message:", message)  # Print the original message
    
    # Encrypt the message
    encrypted_message = encrypt(message, public_key)  # Encrypt the message using the public key
    print("Encrypted Message:", encrypted_message)  # Print the encrypted message
    
    # Decrypt the message
    decrypted_message = decrypt(encrypted_message, private_key)  # Decrypt the message using the private key
    print("Decrypted Message:", decrypted_message)  # Print the decrypted message

if __name__ == "__main__":
    main()  # Call the main function if this script is executed


Enter the message to encrypt: farah
Original Message: farah
Encrypted Message: 23543286258720972048859101058671368301675671054507220061733832833964216625404371806071864094974405014792282462915460794530862170988737272215612557938355000959597652703786424942405473386885330825681196539081393589032502872718403567431793287717308111170543603803881992116407383011437961893137018101960849182434
Decrypted Message: farah


# Double Encryption

In [27]:
def double_encrypt(plaintext, sender_private_key, receiver_public_key):
    """
    Encrypts the plaintext first with the sender's private key, then with the receiver's public key.
    """
    first_encryption = encrypt(plaintext, sender_private_key)  # Encrypt the plaintext with the sender's private key
    double_encryption = encrypt(str(first_encryption), receiver_public_key)  # Encrypt the result with the receiver's public key
    return double_encryption  # Return the double encrypted message


# Double Decryption

In [28]:
def double_decrypt(ciphertext, receiver_private_key, sender_public_key):
    """
    Decrypts the ciphertext first with the receiver's private key, then with the sender's public key.
    """
    # Decrypt the ciphertext with the receiver's private key
    first_decryption = decrypt(ciphertext, receiver_private_key)

    # Check if the result of the first decryption is a string
    if isinstance(first_decryption, str):
        # If it's a string, remove non-digit characters and convert to an integer
        first_decryption_int = int(''.join(filter(str.isdigit, first_decryption)))
    else:
        # If it's already an integer, use it directly
        first_decryption_int = first_decryption

    # Decrypt the integer result of the first decryption with the sender's public key
    double_decryption = decrypt(first_decryption_int, sender_public_key)

    # Return the final decrypted message
    return double_decryption


#  Putting It All Together

In [29]:
def main():
    # Generate public and private keys for both sender and receiver
    sender_public_key, sender_private_key = generate_keys(bits=512)
    receiver_public_key, receiver_private_key = generate_keys(bits=512)
    
    # Get message from user
    message = input("Enter the message to encrypt: ")
    print("Original Message:", message)
    
    # Double encrypt the message
    encrypted_message = double_encrypt(message, sender_private_key, receiver_public_key)
    print("Double Encrypted Message:", encrypted_message)
    
    # Double decrypt the message
    decrypted_message = double_decrypt(encrypted_message, receiver_private_key, sender_public_key)
    print("Double Decrypted Message:", decrypted_message)

if __name__ == "__main__":
    main()

Enter the message to encrypt: farah
Original Message: farah
Double Encrypted Message: 99065940721691397498380719656665333698210203280315285730742003487028909555327070592000107368761246840817626692470550549238621207739017985233896885729708458193138057913296542149130438760724822739336265471538169936632357493471344179448605983305771959579877692484475234986383353803506985704801825205518256354616
Double Decrypted Message: 4833175881105169871927187952582872038422726739822746372544105128982521430693548398277555563801101754265890129026871157807643554380481669457810119327221358711663499306132309514651897952243089191725920605740776585771808619543554833468188962830633941859797839956448594616311499241855309616128814027877558392308
